# `Troll Tweets Using NLP`

# <font color=red>Mr Fugu Data Science</font>

# (◕‿◕✿)

**`Help Support The Channel: Patreon @mrfugudatasci`**

**`Or Consider: Buy Me A Coffee @mrfugudatasci`**

In [210]:
# %%html
# <iframe src="https://giphy.com/embed/WV9MMJsicOOGxl2rE6" width="426" height="480" frameBorder="0" class="giphy-embed" allowFullScreen></iframe><p><a href="https://giphy.com/gifs/memecandy-WV9MMJsicOOGxl2rE6">via GIPHY</a></p>

In [3]:
import pickle                            # open our file
import pandas as pd
import os
import re

# plotting image with word cloud
from PIL import Image                    # for our images
import numpy as np                       # convert image to np arrays
from wordcloud import WordCloud,ImageColorGenerator,STOPWORDS # generate word cloud
import matplotlib.pyplot as plt          # plotting

# nlp
import nltk                              # text processing
from nltk import word_tokenize           # split sentence into list of words
from nltk.corpus import stopwords        # remove: and,it,i,etc
import string                            # remove punctuation
from collections import defaultdict      # dictionary with values as lists
from nltk.stem import WordNetLemmatizer  # remove word endings etc


In [4]:
# Connect to Kaggle Api

from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()

In [5]:
# all datasets with troll keyword:

api.dataset_list(search='Troll')

[vikasg/russian-troll-tweets,
 fivethirtyeight/russian-troll-tweets,
 dataturks/dataset-for-detection-of-cybertrolls,
 tannergi/microcontroller-detection,
 kapastor/democratvsrepublicantweets,
 mohammadqahtani/trolley-dilemma,
 paultimothymooney/russian-political-influence-campaigns,
 ixioixi/3-million-russian-troll-tweets-538,
 kkairu/nbc-russian-troll-tweets,
 tannergi/microcontroller-segmentation,
 wadhasnalhamdan/date-fruit-image-dataset-in-controlled-environment,
 charcoal/controllers,
 realc9der/datasets-for-controlled-over-fittinghubmap,
 kani14am/tweets1,
 hollyhetherington/russian-troll-tweets,
 omermetinn/tweets-about-the-top-companies-from-2015-to-2020,
 kishor1123/nnkdataset,
 aellatif/troll,
 jnunezp19/microcontroller-detection,
 dipinpradeep/microcontroller-detection]

In [6]:
# files in this dataset: total of 9 csv
api.dataset_list_files('fivethirtyeight/russian-troll-tweets').files

[IRAhandle_tweets_7.csv,
 IRAhandle_tweets_3.csv,
 IRAhandle_tweets_8.csv,
 IRAhandle_tweets_1.csv,
 IRAhandle_tweets_5.csv,
 IRAhandle_tweets_2.csv,
 IRAhandle_tweets_9.csv,
 IRAhandle_tweets_4.csv,
 IRAhandle_tweets_6.csv]

# `Create Folder to Store Troll Tweets Data`

In [7]:
import os

# define the name of the directory to be created
path = os.getcwd()+"/troll_tweets"

try:
    os.mkdir(path)
except OSError:
    print ("Creation of the directory %s failed" % path)
else:
    print ("Successfully created the directory %s " % path)

Successfully created the directory /Users/zatoichi59/Desktop/troll_tweets 


# `Get all of the datasets reflecting our search:`

`api.dataset_download_files('file you want',
        'location to send file')`


In [8]:
# Download dataset: you have an 'onwer/file_name'

api.dataset_download_files('fivethirtyeight/russian-troll-tweets',
        '/Users/zatoichi59/Desktop/troll_tweets')

# you can download unzipped as well

In [9]:
# find what is inside the new folder from downloaded data:
troll_tweets_=!ls /Users/zatoichi59/Desktop/troll_tweets
troll_tweets_

['russian-troll-tweets.zip']

In [10]:
def os_dir_search(file):
    u=[]
    for p,n,f in os.walk(os.getcwd()):
        
        for a in f:
            a = str(a)
            if a.endswith(file): # can be (.csv) or a file like I did and search 
#                 print(a)
#                 print(p)
                t=p
    return t

os_dir_search(troll_tweets_[0])

'/Users/zatoichi59/Desktop/troll_tweets01'

In [11]:
from zipfile import ZipFile
# specifying the zip file name 

file_name=os_dir_search(troll_tweets_[0])+ '/' + troll_tweets_[0]
  
# opening the zip file in READ mode 
with ZipFile(file_name, 'r') as zip: 
    # printing all the contents of the zip file 
    zip.printdir() 
  
    # extracting all the files 
    print('Extracting all the files now...') 
    zip.extractall('/Users/zatoichi59/Desktop/troll_tweets') 
    print('Done!')



File Name                                             Modified             Size
IRAhandle_tweets_1.csv                         2019-10-03 00:12:48     87874842
IRAhandle_tweets_2.csv                         2019-10-03 00:12:56     89384804
IRAhandle_tweets_3.csv                         2019-10-03 00:13:02     87696348
IRAhandle_tweets_4.csv                         2019-10-03 00:13:10     87771344
IRAhandle_tweets_5.csv                         2019-10-03 00:13:18     79412644
IRAhandle_tweets_6.csv                         2019-10-03 00:13:24     77962778
IRAhandle_tweets_7.csv                         2019-10-03 00:13:32     82408170
IRAhandle_tweets_8.csv                         2019-10-03 00:13:38     85947534
IRAhandle_tweets_9.csv                         2019-10-03 00:13:46      9444208
Extracting all the files now...
Done!


In [12]:
print('Location of my Insurance Folder: ',os_dir_search(troll_tweets_[0]))
print('-------------------------------')
print('Files in Insurance Folder: \n',os.listdir(os_dir_search(troll_tweets_[0])))

Location of my Insurance Folder:  /Users/zatoichi59/Desktop/troll_tweets01
-------------------------------
Files in Insurance Folder: 
 ['IRAhandle_tweets_1.csv', 'IRAhandle_tweets_2.csv', 'IRAhandle_tweets_3.csv', 'IRAhandle_tweets_4.csv', 'IRAhandle_tweets_5.csv', 'IRAhandle_tweets_6.csv', 'IRAhandle_tweets_7.csv', 'IRAhandle_tweets_8.csv', 'IRAhandle_tweets_9.csv', 'russian-troll-tweets.zip']


In [13]:
# combine the files into a list

my_dir=os_dir_search(troll_tweets_[0])
trolls_=[]
for file in os.listdir(my_dir):
    trolls_.append(file)

# pd.read_csv(trolls_[2]).head()

In [14]:
# Create DF from each file:

k=[]
for i in trolls_[:-1]: # last file is .zip exclude it
    # read each csv into DF
    kaggle_tweets=pd.read_csv(os_dir_search(troll_tweets_[0])+'/'+str(i))
    # save all DF's into list
    k.append(kaggle_tweets)


In [15]:
# Almost 3 Million Troll Tweets, combined from 9 files:

mega_troll_df=pd.concat(k,axis=0,ignore_index=True)

mega_troll_df.tail()

,external_author_id,author,content,region,language,publish_date,harvested_date,following,followers,updates,post_type,account_type,new_june_2018,retweet,account_category
2973366,3.093874e+09,_YOUR_LIFESTYLE,Санкционный пинг-понг http://t.co/Jz06cndR8q,Unknown,Uzbek,9/7/2015 1:31,9/7/2015 1:31,60,4,412,NaN,Russian,0,0,NonEnglish
2973367,3.093874e+09,_YOUR_LIFESTYLE,Горизонтальный кальян NanoSmoke http://t.co/rL...,Unknown,Russian,9/8/2015 14:49,9/8/2015 14:49,60,4,416,NaN,Russian,0,0,NonEnglish
2973368,3.093874e+09,_YOUR_LIFESTYLE,Фотограф Kai-Hendrik Schroeder http://t.co/q0x...,Unknown,Russian,9/9/2015 18:52,9/9/2015 18:52,60,4,418,NaN,Russian,0,0,NonEnglish
2973369,3.093874e+09,_YOUR_LIFESTYLE,К чему стремится компания Netflix (США) http:/...,Unknown,Russian,9/9/2015 22:26,9/9/2015 22:27,60,4,419,NaN,Russian,0,0,NonEnglish
2973370,3.093874e+09,_YOUR_LIFESTYLE,В Петербурге появилось вегетарианское кафе в к...,Unknown,Russian,9/9/2015 8:17,9/9/2015 8:17,60,4,417,NaN,Russian,0,0,NonEnglish


In [16]:
mega_troll_df.columns

Index(['external_author_id', 'author', 'content', 'region', 'language',
       'publish_date', 'harvested_date', 'following', 'followers', 'updates',
       'post_type', 'account_type', 'new_june_2018', 'retweet',
       'account_category'],
      dtype='object')

In [17]:
# Check for NA's

mega_troll_df.isna().sum().sort_values(ascending=False)

post_type             1662425
region                   8843
account_type              363
external_author_id          4
content                     1
account_category            0
retweet                     0
new_june_2018               0
updates                     0
followers                   0
following                   0
harvested_date              0
publish_date                0
language                    0
author                      0
dtype: int64

In [18]:
mega_troll_df['account_type'].unique()

array(['Right', 'Russian', 'Koch', 'Italian', 'left', '?', 'German',
       'Spanish', 'Hashtager', 'Arabic', 'local', 'Commercial', 'French',
       'Ukranian', 'ZAPOROSHIA', 'news', 'right', 'Uzbek', 'Ebola ', nan,
       'Portuguese'], dtype=object)

In [19]:
mega_troll_df['post_type'].unique()

array([nan, 'RETWEET', 'QUOTE_TWEET'], dtype=object)

In [20]:
mega_troll_df['region'].unique()

array(['Unknown', nan, 'United States', 'Italy', 'United Arab Emirates',
       'Japan', 'Israel', 'Azerbaijan', 'Egypt', 'United Kingdom',
       'Russian Federation', 'Turkey', 'Iraq', 'Germany', 'France',
       'Ukraine', 'Serbia', 'Hong Kong', 'Austria', 'Belarus', 'Malaysia',
       'Spain', 'Samoa', 'India', 'Afghanistan', 'Saudi Arabia',
       'Iran, Islamic Republic of', 'Mexico', 'Canada', 'Greece',
       'Czech Republic', 'Finland', 'Latvia', 'Estonia', 'Sweden',
       'Denmark', 'Switzerland'], dtype=object)

In [21]:
mega_troll_df['region'].value_counts()[:10]

United States           2055882
Unknown                  572767
Azerbaijan               100755
United Arab Emirates      74908
Russian Federation        37637
Belarus                   29619
Germany                   27192
United Kingdom            18062
Italy                     13494
Iraq                      11219
Name: region, dtype: int64

In [22]:
mega_troll_df['account_type'].value_counts()[:10]

Russian       721191
Right         718619
local         460197
left          427811
Hashtager     241827
news          139097
Commercial    122582
German         91851
Italian        15899
?              13542
Name: account_type, dtype: int64

In [23]:
mega_troll_df['language'].value_counts()[:10]


# .value_count(normalize=True) if you want proportions

English               2128963
Russian                624124
German                  87171
Ukrainian               39361
Italian                 18254
Serbian                  9615
Uzbek                    9491
Bulgarian                9458
LANGUAGE UNDEFINED       8325
Arabic                   7595
Name: language, dtype: int64

In [24]:
mega_troll_df['author'].value_counts()[:20]

# .plot(kind='bar') if you want to visualize

EXQUOTE            59652
SCREAMYMONKEY      44041
WORLDNEWSPOLI      36974
AMELIEBALDWIN      35371
TODAYPITTSBURGH    33602
SPECIALAFFAIR      32588
SEATTLE_POST       30800
FINDDIET           29038
KANSASDAILYNEWS    28890
ROOMOFRUMOR        28360
DAILYSANFRAN       28076
WORLDOFHASHTAGS    23724
COVFEFENATIONUS    23544
POLITICS_T0DAY     23034
RIAFANRU           22886
JENN_ABRAMS        22861
CHICAGODAILYNEW    22236
FUNDDIET           20192
BERLINBOTE         19748
CHESPLAYSCHESS     19216
Name: author, dtype: int64

# `Start Cleaning:`

In [25]:
# there were a large number of nan's needing to be replaced:
mega_troll_df['post_type'].fillna(value='Other_Tweet',
                                        inplace = True)

mega_troll_df['region'].fillna(value='Unknown',
                                            inplace = True)

mega_troll_df['account_type']=mega_troll_df['account_type'].\
replace('?','Unknown')


mega_troll_df['account_type'].fillna(value='Unknown',
                                            inplace = True)

In [26]:
mega_troll_df['account_type'].unique()

array(['Right', 'Russian', 'Koch', 'Italian', 'left', 'Unknown', 'German',
       'Spanish', 'Hashtager', 'Arabic', 'local', 'Commercial', 'French',
       'Ukranian', 'ZAPOROSHIA', 'news', 'right', 'Uzbek', 'Ebola ',
       'Portuguese'], dtype=object)

In [27]:
mega_troll_df.isna().sum().sort_values(ascending=False)

external_author_id    4
content               1
account_category      0
retweet               0
new_june_2018         0
account_type          0
post_type             0
updates               0
followers             0
following             0
harvested_date        0
publish_date          0
language              0
region                0
author                0
dtype: int64

In [28]:
# Let's Investigate the Nan values for each of the columns:

# we will drop this useless row:
mega_troll_df[mega_troll_df.loc[:,'content'].isna()==True]

row_to_drop=mega_troll_df[mega_troll_df.loc[:,'content'].\
                          isna()==True].index[0]

mega_troll_df.drop(row_to_drop,axis='index',inplace=True)

In [29]:
# when you investigate the external_author_id: but we will 
# remove this columns!
mega_troll_df[mega_troll_df.loc[:,'external_author_id'].isna()==True]


,external_author_id,author,content,region,language,publish_date,harvested_date,following,followers,updates,post_type,account_type,new_june_2018,retweet,account_category
1095528,NaN,FINDDIET,muvva @__SlaYonce Marissa @marissafabrizi_ Taw...,United States,LANGUAGE UNDEFINED,8/12/2015 11:04,8/12/2015 11:12,3,308,29699,Other_Tweet,Commercial,1,0,Commercial
1096572,NaN,FINDDIET,http://t.co/MKzEwWBPwr Toe @hongtony22 ժɑɾíɑղ ...,United States,LANGUAGE UNDEFINED,8/12/2015 23:08,8/12/2015 23:08,3,331,30648,Other_Tweet,Commercial,1,0,Commercial
1097335,NaN,FINDDIET,http://t.co/wFGUVzURSM Jay .@Jay_zo Nina .@xxo...,United States,LANGUAGE UNDEFINED,8/13/2015 0:18,8/13/2015 0:19,3,334,30756,Other_Tweet,Commercial,1,0,Commercial
1098750,NaN,FINDDIET,http://t.co/U4DFGCaSEF Davi_hoops @comiclyfe_1...,United States,LANGUAGE UNDEFINED,8/13/2015 6:27,8/13/2015 6:27,3,350,31267,Other_Tweet,Commercial,1,0,Commercial


In [30]:
# remove useless columns:

mega_troll_df=mega_troll_df.drop(['external_author_id',
                    'harvested_date','new_june_2018'],axis=1)
mega_troll_df.head()

,author,content,region,language,publish_date,following,followers,updates,post_type,account_type,retweet,account_category
0,10_GOP,"""We have a sitting Democrat US Senator on tria...",Unknown,English,10/1/2017 19:58,1052,9636,253,Other_Tweet,Right,0,RightTroll
1,10_GOP,Marshawn Lynch arrives to game in anti-Trump s...,Unknown,English,10/1/2017 22:43,1054,9637,254,Other_Tweet,Right,0,RightTroll
2,10_GOP,Daughter of fallen Navy Sailor delivers powerf...,Unknown,English,10/1/2017 22:50,1054,9637,255,RETWEET,Right,1,RightTroll
3,10_GOP,JUST IN: President Trump dedicates Presidents ...,Unknown,English,10/1/2017 23:52,1062,9642,256,Other_Tweet,Right,0,RightTroll
4,10_GOP,"19,000 RESPECTING our National Anthem! #StandF...",Unknown,English,10/1/2017 2:13,1050,9645,246,RETWEET,Right,1,RightTroll


# `Start Cleaning Text: Remove Http(s) and replace with empty space`

+ these links are an issue:
    + They can link to bad webpages, spam, or naughty places
    + They are useless for NLP and create useless info

In [31]:
import re

text_wo_http=[]
for i in mega_troll_df['content']:
    re_pattern=r'\b(?:http).+\b'
    re_p=r'http\S+'
    if type(i)==str and re.findall(re_pattern,i):
        text_wo_http.append(re.sub(re_p,'',i))
    else:
        text_wo_http.append(i)
        
len(text_wo_http )

2973370

In [32]:
mega_troll_df['content without http']=text_wo_http

In [33]:
mega_troll_df.columns

Index(['author', 'content', 'region', 'language', 'publish_date', 'following',
       'followers', 'updates', 'post_type', 'account_type', 'retweet',
       'account_category', 'content without http'],
      dtype='object')

In [34]:
# subset columns we will use:

mega_troll_df.tail(7)

,author,content,region,language,publish_date,following,followers,updates,post_type,account_type,retweet,account_category,content without http
2973364,_YOUR_LIFESTYLE,Художник Шон Йоро http://t.co/S11YeVbFN1,Unknown,Russian,9/7/2015 16:01,60,4,415,Other_Tweet,Russian,0,NonEnglish,Художник Шон Йоро
2973365,_YOUR_LIFESTYLE,Как путешествовать бесплатно? Спасать черепах ...,Unknown,Russian,9/7/2015 1:31,60,4,413,Other_Tweet,Russian,0,NonEnglish,Как путешествовать бесплатно? Спасать черепах ...
2973366,_YOUR_LIFESTYLE,Санкционный пинг-понг http://t.co/Jz06cndR8q,Unknown,Uzbek,9/7/2015 1:31,60,4,412,Other_Tweet,Russian,0,NonEnglish,Санкционный пинг-понг
2973367,_YOUR_LIFESTYLE,Горизонтальный кальян NanoSmoke http://t.co/rL...,Unknown,Russian,9/8/2015 14:49,60,4,416,Other_Tweet,Russian,0,NonEnglish,Горизонтальный кальян NanoSmoke
2973368,_YOUR_LIFESTYLE,Фотограф Kai-Hendrik Schroeder http://t.co/q0x...,Unknown,Russian,9/9/2015 18:52,60,4,418,Other_Tweet,Russian,0,NonEnglish,Фотограф Kai-Hendrik Schroeder
2973369,_YOUR_LIFESTYLE,К чему стремится компания Netflix (США) http:/...,Unknown,Russian,9/9/2015 22:26,60,4,419,Other_Tweet,Russian,0,NonEnglish,К чему стремится компания Netflix (США)
2973370,_YOUR_LIFESTYLE,В Петербурге появилось вегетарианское кафе в к...,Unknown,Russian,9/9/2015 8:17,60,4,417,Other_Tweet,Russian,0,NonEnglish,В Петербурге появилось вегетарианское кафе в к...


# `There is a lot going with these!`

+ We have different languages
    + `Googletrans api:` is inconsistent, you will get times where it doesn't even work.
    As of *Jan. 5th 2021*. It was NOT working, I downloaded `pip3 install google_trans_new`
    + This can work, but if you are using Anaconda, it will not.
        + This is because Anaconda doesn't update often I guess.
+ If you are in my predicament, I suggest going in a terminal or your favorite IDE and making a script to parse these if you want.
    + Understand that you will not be able to ensure that the english will be exactly correction. As of yet, I have seen NO one do analysis with translated data... I will see if I can try.

In [45]:
# Replace single and double qoutes which are going to mess us up!
# this is NOT fast code btw
str_repl=[]
for i in mega_troll_df['content without http'].values:    
    re_pat=r'^"|"$'
    re_pat2=r"^'|'$"
#     if type(i)==str and re.findall(re_pat,i):
#         str_repl.append(re.sub(r'^"|"$', '', i))
#         str_repl.append(i.strip('"'))
#     elif type(i)==str and re.findall(re_pat2,i):
#         str_repl.append(i.strip("'"))
#     else:
#     str_repl.append(i.strip('"').strip("'"))
    str_repl.append(i.replace('"','').replace("'",''))

In [46]:
str_repl[:3]

['We have a sitting Democrat US Senator on trial for corruption and youve barely heard a peep from the mainstream media. ~ @nedryun ',
 'Marshawn Lynch arrives to game in anti-Trump shirt. Judging by his sagging pants the shirt should say Lynch vs. belt ',
 'Daughter of fallen Navy Sailor delivers powerful monologue on anthem protests, burns her NFL packers gear.  #BoycottNFL ']

In [48]:
mega_troll_df['content without http']=str_repl

# mega_troll_df.tail()

mega_troll_df.to_csv('troll_tweets_vid02.csv')

# `Split Data By English & Russian:`

In [ ]:
# ENGLISH:

english_tweets=mega_troll_df[mega_troll_df['language']=='English']

# Russian

Russian_tweets=mega_troll_df[mega_troll_df['language']=='Russian']

In [206]:
# Russian_tweets.to_csv('Russian_tweets.csv')

# `Translate Tweets to english:`

+ I haven't seen this done yet for analysis, but decided it may be useful. I think it may not be needed for some forms of analysis also.

**`pip install google_trans_new`**

# <font color=red>Like</font>, Share &

# <font color=red>SUB</font>scribe

**`Help Support The Channel: Patreon @mrfugudatasci`**

**`Or Consider: Buy Me A Coffee @mrfugudatasci`**

# `Citations & Help:`

# ◔̯◔

`Kaggle`

https://medium.com/@jeff.daniel77/accessing-the-kaggle-com-api-with-jupyter-notebook-on-windows-d6f330bc6953

https://github.com/Kaggle/kaggle-api

https://technowhisp.com/kaggle-api-python-documentation/

https://www.kaggle.com/docs/api#interacting-with-datasets

https://www.geeksforgeeks.org/working-zip-files-python/

https://medium.com/analytics-vidhya/fetch-data-from-kaggle-with-python-9154a4c610e3

https://stackabuse.com/creating-and-deleting-directories-with-python/

https://www.kaggle.com/paultimothymooney/exploring-the-kaggle-api

https://stackoverflow.com/questions/50836642/i-get-a-bash-command-not-found-when-trying-to-use-the-kaggle-api

https://github.com/Kaggle/kaggle-api/issues/29

`NLP`

https://stackabuse.com/text-translation-with-google-translate-api-in-python/ (translate txt)

https://medium.com/analytics-vidhya/how-to-translate-text-with-python-9d203139dcf5 (translation options)


`Weird python stuff`

https://opensource.com/article/19/5/python-3-default-mac

https://stackoverflow.com/questions/11331982/how-to-remove-any-url-within-a-string-in-python